In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

In [3]:
def branin(x, y):
    a, b, c, d, e, f = 1.0, 5.1 / (4 * np.pi ** 2), 5 / np.pi, 6, 10, 1 / (8 * np.pi)
    return a * (y - b * x ** 2 + c * x - d) ** 2 + e * (1 - f) * np.cos(x) + e

def easom(x, y):
    return -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi) ** 2 + (y - np.pi) ** 2))

def goldstein_price(x, y):
    term1 = (1 + ((x + y + 1) ** 2) * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2))
    term2 = (30 + ((2*x - 3*y) ** 2) * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return term1 * term2

def six_hump_camel(x, y):
    return (4 - 2.1 * x ** 2 + (x ** 4) / 3) * x ** 2 + x * y + (-4 + 4 * y ** 2) * y ** 2


In [4]:
def select_parents(population, fitness, num_parents):
    fitness_min = np.min(fitness)
    adjusted_fitness = fitness - fitness_min 
    
    probabilities = np.exp(-adjusted_fitness)  
    probabilities += 1e-10 
    probabilities /= np.sum(probabilities)

    best_idx = np.argmin(fitness)  
    parents = [population[best_idx]]  

    num_remaining_parents = num_parents - 1 
    if num_remaining_parents > 0:
        remaining_indices = np.random.choice(len(population), num_remaining_parents, replace=False, p=probabilities)
        parents.extend(population[remaining_indices])

    return np.array(parents)

def arithmetic_crossover(p1, p2):
    alpha = np.random.rand()
    child1 = alpha * p1 + (1 - alpha) * p2
    child2 = (1 - alpha) * p1 + alpha * p2
    return child1, child2

def adaptive_mutation(offspring, generation, max_generations, mutation_rate, extra_mutation=0):
    if extra_mutation:
        mutation_strength = extra_mutation
    else:
        mutation_strength = 0.1 * (1 - generation / max_generations) 

    mutations = np.random.uniform(-mutation_strength, mutation_strength, offspring.shape)
    offspring += mutations * (np.random.rand(*offspring.shape) < mutation_rate)
    return offspring

In [5]:

def genetic_algorithm(func, bounds, pop_size=50, generations=100, mutation_rate=0.7, crossover_rate=0.8, stagnation_limit=10, epsilon=1e-8):
    population = np.random.uniform(bounds[:, 0], bounds[:, 1], (pop_size, 2))
    best_points = []  
    all_populations = []  
    best_fitness = float('inf')
    stagnation_counter = 0  

    for gen in range(generations):
        fitness = np.array([func(*ind) for ind in population])

        best_idx = np.argmin(fitness)
        current_best_fitness = fitness[best_idx]
        best_points.append(tuple(population[best_idx]))
        all_populations.append(population.copy())

        if abs(current_best_fitness - best_fitness) < epsilon:
            stagnation_counter += 1
        else:
            stagnation_counter = 0  

        best_fitness = current_best_fitness

        if stagnation_counter >= 5:
            extra_mutation = np.max(bounds) * 0.1
        else:
            extra_mutation = 0

        if stagnation_counter >= stagnation_limit:
            print(f"Early stopping at generation {gen} due to minimal progress.")
            break

        selected = select_parents(population, fitness, pop_size // 2)
        offspring = []

        for _ in range(pop_size // 2):
            p1, p2 = selected[np.random.choice(len(selected), 2, replace=False)]
            
            if np.random.rand() < crossover_rate:
                child1, child2 = arithmetic_crossover(p1, p2)  
                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.append(p1)
                offspring.append(p2)

        offspring = np.array(offspring)
        offspring = adaptive_mutation(offspring, gen, generations, mutation_rate, extra_mutation)  
        offspring = np.clip(offspring, bounds[:, 0], bounds[:, 1])

        population = np.vstack((selected, offspring))

        if gen % 10 == 0:
            print(f"Gen {gen}: Best Fitness = {best_fitness:.6f}")

    return best_points, all_populations

In [6]:
bounds = np.array([[-5, 10], [0, 15]])

best_points, all_populations = genetic_algorithm(branin, bounds, pop_size=50, generations=100)

best_solution = best_points[-1]
best_value = branin(*best_solution)

best_solution, best_value

Gen 0: Best Fitness = 1.087695
Gen 10: Best Fitness = 0.399568
Gen 20: Best Fitness = 0.398215
Early stopping at generation 27 due to minimal progress.


((np.float64(9.419807602276048), np.float64(2.485272474493543)),
 np.float64(0.3982151111477048))